In [1]:
import pandas as pd
import re

In [5]:
# Load dataframes
current = pd.read_csv('data/Current_Sheet.csv')
emissions_limited = pd.read_csv('data/Emissions_Limited_2040.csv')
business_as_usual= pd.read_csv('data/Business_as_Usual_2080.csv')
bau_plus_1degree = pd.read_csv('data/BAU_plus_1degree_2080.csv')

In [ ]:
def compare_taxon_lists(df1, df2, Taxon="Taxon"):
    """
    Compare the taxon lists of two datasets.

    Parameters:
    df1 (pd.DataFrame): First dataframe to compare.
    df2 (pd.DataFrame): Second dataframe to compare.currrent
    
    Returns:
    dict: A dictionary with keys 'unique_to_df1', 'unique_to_df2', and 'common' 
          containing taxon lists unique to each dataframe and common to both.
    """
    #remove duplicates
    taxon_df1 = set(df1[Taxon].dropna().unique())
    taxon_df2 = set(df2[Taxon].dropna().unique())

    #compare dataframes
    unique_to_df1 = list(taxon_df1 - taxon_df2)
    unique_to_df2 = list(taxon_df2 - taxon_df1)
    common = list(taxon_df1 & taxon_df2)
    
    return {
        'unique_to_df1': unique_to_df1,
        'unique_to_df2': unique_to_df2,
        'common': common
    }


In [ ]:
#input to dataframes names to compare 'Taxon' items
result = compare_taxon_lists(current, business_as_usual)

print(f"""Test results:
Unique to df1 ({len(result['unique_to_df1'])}) 
Unique to df2 ({len(result['unique_to_df2'])})
Common ({len(result['common'])})
""")


In [ ]:
#input name of dataframe to count
data = current

#Count the occurrences
taxon_counts = data['Taxon'].value_counts()

#Find duplicates > 1)
duplicates = taxon_counts[taxon_counts > 1]

#Create a DataFrame of duplicates
duplicates_df = duplicates.reset_index()
duplicates_df.columns = ['Taxon', 'Count']

#Display
duplicates_df


# Observation Records

### The observation records that are used to extrapolate range in the CAT model are based on a pooling data 
### from several plant database (Plant Search, GBIF, GBIG-BGCI, Urban Plants). The number of observations in the 
### given database has been entered as a string value "MODEL=#" where MODEL= is the model name and # is the number
### of observations. Three of the sheets from the CAT report had 5 columns with observation reports, while the fourth
### has only three observation columns after removing 2 erred columns earlier in this analysis. This data structure
### is untidy.

### to tidy the data  well apply a form of 'one-hot coding' whereby the model name becomes a unique column and the number
### of observation are recorded in the field when the model was used. This style preserves the boolean nature of one-hot, 
### as any value other than nan/0 would be TRUE that said model was used, all the while preserving the numerical value
### of the count

In [16]:
#name of data frame to 'one-hot' encode
original_df = bau_plus_1degree

# Extract all unique model types from obs_record columns
obs_record_columns = [col for col in original_df.columns if col.startswith('obs_record')]
model_types = set()

# Function to extract model type from a string
def extract_model_type(obs_string):
    if isinstance(obs_string, str):
        return obs_string.split('=')[0]
    return None

# Collecting all unique model types
for col in obs_record_columns:
    original_df[col].dropna().apply(lambda x: model_types.add(extract_model_type(x)))

# Create new columns for each model type with default NaN values
for model in model_types:
    original_df[model] = pd.NA

# Populate the new columns with observation counts
def populate_model_counts(row):
    for col in obs_record_columns:
        if pd.notna(row[col]):
            model, count = row[col].split('=')
            row[model] = int(count)
    return row

encoded_df = original_df.apply(populate_model_counts, axis=1)

# Dropping original obs_record columns
encoded_df.drop(columns=obs_record_columns, inplace=True)



In [17]:
encoded_df

,Taxon,temperature,climate_rating,mean_annual_ temp_celcius,mean_hottest_month_celcius,mean_coldest_quarter_celsius,mean_annual_precipitation mm/year,mean_driest_quarter mm/qtr,tree_on_CAT,PLANTSEARCH,URBANPLANTS,GBIF_CURRENT,GBIF_BGCI
0,Arenaria purpurascens,15.3,0,9.75833,23.0000,2.483330,797.000,148.0000,False,36,<NA>,<NA>,<NA>
1,Aria alnifolia,15.3,0,10.39580,25.4500,0.825000,1050.500,127.5000,False,2,<NA>,<NA>,<NA>
2,Berberis duclouxiana,15.3,0,10.54580,24.5000,3.333330,710.000,146.0000,False,2,<NA>,<NA>,<NA>
3,Cotoneaster hedegaardii,15.3,0,10.13750,22.0500,4.208330,811.500,161.0000,False,2,<NA>,<NA>,<NA>
4,Phacelia sericea,15.3,0,9.61041,23.3500,0.591667,754.500,143.0000,False,21,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3821,Vitex agnus-castus,15.3,11,15.44750,29.6830,8.186660,637.000,89.5000,True,211,17,7669,2556
3822,Wollemia nobilis,15.3,11,14.49890,27.1435,7.409830,760.625,134.1250,True,132,4,28,54
3823,Yucca baccata,15.3,11,14.20140,31.8833,5.256670,466.833,61.3333,True,71,<NA>,1638,456
3824,Zanthoxylum oxyphyllum,15.3,11,13.92920,23.9500,7.322220,941.833,83.1667,True,7,<NA>,17,46


In [18]:
#save data frame to csv file

#change file name and path as required
output_file_path = 'data/BAU_plus_1degree_2080_encoded.csv'
encoded_df.to_csv(output_file_path, index=False);
